In [ ]:
# -*- coding: utf-8 -*-
import os, sys, random, io, urllib
from datetime import datetime

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

import pandas as pd
import random as rd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')
from IPython.display import Image, display

In [ ]:
USE_CUDA = (torch.backends.cudnn.version() != None)

seed_value = 1234
rd.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
if USE_CUDA:
    torch.cuda.manual_seed(seed_value)

if not os.path.exists('./data'): os.makedirs('./data')
if not os.path.exists('./models'): os.makedirs('./models')

ori_dataset = pd.read_csv('./data/fraud_dataset_v2.csv')
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] Transactional dataset of {} rows and {} columns loaded'.format(now, ori_dataset.shape[0], ori_dataset.shape[1]))

In [ ]:
ori_dataset.head(10)

In [ ]:
ori_dataset.label.value_counts()

In [ ]:
label = ori_dataset.pop('label')

categorical_attr_names = ['KTOSL', 'PRCTR', 'BSCHL', 'HKONT', 'BUKRS', 'WAERS']

ori_dataset_categ_transformed = pd.get_dummies(ori_dataset[categorical_attr_names])
ori_dataset_categ_transformed.head(10)

In [ ]:
numeric_attr_names = ['DMBTR', 'WRBTR']

numeric_attr = ori_dataset[numeric_attr_names] + 1e-4
numeric_attr = numeric_attr.apply(np.log)

ori_dataset_numeric_attr = (numeric_attr - numeric_attr.min()) / (numeric_attr.max() - numeric_attr.min())

ori_subset_transformed = pd.concat([ori_dataset_categ_transformed, ori_dataset_numeric_attr], axis = 1)
ori_subset_transformed.shape

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()

        self.map_L1 = nn.Linear(input_size, hidden_size[0], bias=True)
        nn.init.xavier_uniform_(self.map_L1.weight)
        nn.init.constant_(self.map_L1.bias, 0.0)
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L2 = nn.Linear(hidden_size[0], hidden_size[1], bias=True)
        nn.init.xavier_uniform_(self.map_L2.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L3 = nn.Linear(hidden_size[1], hidden_size[2], bias=True)
        nn.init.xavier_uniform_(self.map_L3.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L4 = nn.Linear(hidden_size[2], hidden_size[3], bias=True)
        nn.init.xavier_uniform_(self.map_L4.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_R4 = torch.nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L5 = nn.Linear(hidden_size[3], hidden_size[4], bias=True)
        nn.init.xavier_uniform_(self.map_L5.weight)
        nn.init.constant_(self.map_L5.bias, 0.0)
        self.map_R5 = torch.nn.LeakyReLU(negative_slope=0.4, inplace=True)

    def forward(self, x):
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_R4(self.map_L4(x))
        x = self.map_R5(self.map_L5(x))

        return x

encoder_train = Encoder(input_size=ori_subset_transformed.shape[1], hidden_size=[256, 64, 16, 4, 2])

if USE_CUDA:
    encoder_train = encoder_train.cuda()

now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] encoder-generator architecture:\n\n{}\n'.format(now, encoder_train))

class Decoder(nn.Module):
    def __init__(self, output_size, hidden_size):
        super(Decoder, self).__init__()

        self.map_L1 = nn.Linear(hidden_size[0], hidden_size[1], bias=True)
        nn.init.xavier_uniform_(self.map_L1.weight)
        nn.init.constant_(self.map_L1.bias, 0.0)
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L2 = nn.Linear(hidden_size[1], hidden_size[2], bias=True)
        nn.init.xavier_uniform_(self.map_L2.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L3 = nn.Linear(hidden_size[2], hidden_size[3], bias=True)
        nn.init.xavier_uniform_(self.map_L3.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L4 = nn.Linear(hidden_size[3], hidden_size[4], bias=True)
        nn.init.xavier_uniform_(self.map_L4.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_R4 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L5 = nn.Linear(hidden_size[4], output_size, bias=True)
        nn.init.xavier_uniform_(self.map_L5.weight)
        nn.init.constant_(self.map_L5.bias, 0.0)
        self.map_S5 = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_R4(self.map_L4(x))
        x = self.map_S5(self.map_L5(x))

        return x

decoder_train = Decoder(output_size=ori_subset_transformed.shape[1], hidden_size=[2, 4, 16, 64, 256])

if USE_CUDA:
    decoder_train = decoder_train.cuda()

now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] decoder architecture:\n\n{}\n'.format(now, decoder_train))

class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Discriminator, self).__init__()

        self.map_L1 = nn.Linear(input_size, hidden_size[0], bias=True)
        nn.init.xavier_uniform_(self.map_L1.weight)
        nn.init.constant_(self.map_L1.bias, 0.0)
        self.map_R1 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L2 = nn.Linear(hidden_size[0], hidden_size[1], bias=True)
        nn.init.xavier_uniform_(self.map_L2.weight)
        nn.init.constant_(self.map_L2.bias, 0.0)
        self.map_R2 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L3 = nn.Linear(hidden_size[1], hidden_size[2], bias=True)
        nn.init.xavier_uniform_(self.map_L3.weight)
        nn.init.constant_(self.map_L3.bias, 0.0)
        self.map_R3 = nn.LeakyReLU(negative_slope=0.4, inplace=True)

        self.map_L4 = nn.Linear(hidden_size[2], output_size, bias=True)
        nn.init.xavier_uniform_(self.map_L4.weight)
        nn.init.constant_(self.map_L4.bias, 0.0)
        self.map_S4 = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.map_R1(self.map_L1(x))
        x = self.map_R2(self.map_L2(x))
        x = self.map_R3(self.map_L3(x))
        x = self.map_S4(self.map_L4(x))

        return x

discriminator_train = Discriminator(input_size=2, hidden_size=[256, 16, 4], output_size=1)

if USE_CUDA:
    discriminator_train = discriminator_train.cuda()

now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] discriminator architecture:\n\n{}\n'.format(now, discriminator_train))

In [ ]:
reconstruction_criterion_categorical = nn.BCELoss(reduction='mean')
reconstruction_criterion_numeric = nn.MSELoss(reduction='mean')

if USE_CUDA:
    reconstruction_criterion_categorical = reconstruction_criterion_categorical.cuda()
    reconstruction_criterion_numeric = reconstruction_criterion_numeric.cuda()

learning_rate_enc = 1e-3
learning_rate_dec = 1e-3

encoder_optimizer = optim.Adam(encoder_train.parameters(), lr=learning_rate_enc)
decoder_optimizer = optim.Adam(decoder_train.parameters(), lr=learning_rate_dec)

discriminator_criterion = nn.BCELoss()

if USE_CUDA:
    discriminator_criterion = discriminator_criterion.cuda()

learning_rate_dis_z = 1e-5

discriminator_optimizer = optim.Adam(discriminator_train.parameters(), lr=learning_rate_dis_z)

In [ ]:
tau = 5
radius = 0.8
sigma = 0.01
dim = 2

x_centroid = (radius * np.sin(np.linspace(0, 2 * np.pi, tau, endpoint=False)) + 1) / 2
y_centroid = (radius * np.cos(np.linspace(0, 2 * np.pi, tau, endpoint=False)) + 1) / 2

mu_gauss = np.vstack([x_centroid, y_centroid]).T

samples_per_gaussian = 100000

for i, mu in enumerate(mu_gauss):
    if i == 0:
        z_continous_samples_all = np.random.normal(mu, sigma, size=(samples_per_gaussian, dim))
    else:
        z_continous_samples = np.random.normal(mu, sigma, size=(samples_per_gaussian, dim))
        z_continous_samples_all = np.vstack([z_continous_samples_all, z_continous_samples])

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
ax.scatter(z_continous_samples_all[:, 0], z_continous_samples_all[:, 1], c='C0', marker="o", edgecolors='w', linewidth=0.5)
ax.set_xlabel('$z_1$')
ax.set_ylabel('$z_2$')
ax.set_title('Prior Latent Space Distribution $p(z)$')

In [ ]:
num_epochs = 10
mini_batch_size = 128

torch_dataset = torch.from_numpy(ori_subset_transformed.values).float()

dataloader = DataLoader(torch_dataset, batch_size=mini_batch_size, shuffle=True, num_workers=0)

if USE_CUDA:
    dataloader = DataLoader(torch_dataset.cuda(), batch_size=mini_batch_size, shuffle=True)

epoch_reconstruction_losses = []
epoch_discriminator_losses = []
epoch_generator_losses = []

In [ ]:
for epoch in range(num_epochs):
    mini_batch_count = 0

    batch_reconstruction_losses = 0.0
    batch_discriminator_losses = 0.0
    batch_generator_losses = 0.0

    encoder_train.train()
    decoder_train.train()
    discriminator_train.train()

    start_time = datetime.now()

    for mini_batch_data in dataloader:
        mini_batch_count += 1

        if USE_CUDA:
            mini_batch_torch = torch.cuda.FloatTensor(mini_batch_data)
        else:
            mini_batch_torch = torch.FloatTensor(mini_batch_data)

        encoder_train.zero_grad()
        decoder_train.zero_grad()
        discriminator_train.zero_grad()

        # =================== reconstruction phase =====================

        z_sample = encoder_train(mini_batch_torch)
        mini_batch_reconstruction = decoder_train(z_sample)

        batch_cat = mini_batch_torch[:, :ori_dataset_categ_transformed.shape[1]]
        batch_num = mini_batch_torch[:, ori_dataset_categ_transformed.shape[1]:]

        rec_batch_cat = mini_batch_reconstruction[:, :ori_dataset_categ_transformed.shape[1]]
        rec_batch_num = mini_batch_reconstruction[:, ori_dataset_categ_transformed.shape[1]:]

        rec_error_cat = reconstruction_criterion_categorical(input=rec_batch_cat, target=batch_cat)
        rec_error_num = reconstruction_criterion_numeric(input=rec_batch_num, target=batch_num)

        reconstruction_loss = rec_error_cat + rec_error_num

        reconstruction_loss.backward()

        batch_reconstruction_losses += reconstruction_loss.item()

        decoder_optimizer.step()
        encoder_optimizer.step()

        # =================== regularization phase =====================
        # =================== discriminator training ===================

        discriminator_train.eval()

        z_target_batch = z_continous_samples_all[random.sample(range(0, z_continous_samples_all.shape[0]), mini_batch_size),:]

        z_target_batch = torch.FloatTensor(z_target_batch)

        if USE_CUDA:
            z_target_batch = z_target_batch.cuda()

        z_fake_gauss = encoder_train(mini_batch_torch)

        d_real_gauss = discriminator_train(z_target_batch)
        d_fake_gauss = discriminator_train(z_fake_gauss)

        d_real_gauss_target = torch.FloatTensor(torch.ones(d_real_gauss.shape))
        d_fake_gauss_target = torch.FloatTensor(torch.zeros(d_fake_gauss.shape))

        if USE_CUDA:
            d_real_gauss_target = d_real_gauss_target.cuda()
            d_fake_gauss_target = d_fake_gauss_target.cuda()

        discriminator_loss_real = discriminator_criterion(target=d_real_gauss_target, input=d_real_gauss)
        discriminator_loss_fake = discriminator_criterion(target=d_fake_gauss_target, input=d_fake_gauss)

        discriminator_loss = discriminator_loss_fake + discriminator_loss_real

        discriminator_loss.backward()

        batch_discriminator_losses += discriminator_loss.item()

        discriminator_optimizer.step()

        # =================== regularization phase =====================
        # =================== generator training =======================

        encoder_train.train()
        encoder_train.zero_grad()

        z_fake_gauss = encoder_train(mini_batch_torch)

        d_fake_gauss = discriminator_train(z_fake_gauss)

        d_fake_gauss_target = torch.FloatTensor(torch.ones(d_fake_gauss.shape))

        if USE_CUDA:
            d_fake_gauss_target = d_fake_gauss_target.cuda()

        generator_loss = discriminator_criterion(target=d_fake_gauss_target, input=d_fake_gauss)

        batch_generator_losses += generator_loss.item()

        generator_loss.backward()

        encoder_optimizer.step()

    epoch_reconstruction_loss = batch_reconstruction_losses / mini_batch_count
    epoch_reconstruction_losses.extend([epoch_reconstruction_loss])

    epoch_discriminator_loss = batch_discriminator_losses / mini_batch_count
    epoch_discriminator_losses.extend([epoch_discriminator_loss])

    epoch_generator_loss = batch_generator_losses / mini_batch_count
    epoch_generator_losses.extend([epoch_generator_loss])

    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG TRAIN {}] epoch: {:04}/{:04}, reconstruction loss: {:.4f}'.format(now, epoch + 1, num_epochs, epoch_reconstruction_loss))
    print('[LOG TRAIN {}] epoch: {:04}/{:04}, discriminator loss: {:.4f}'.format(now, epoch + 1, num_epochs, epoch_discriminator_loss))
    print('[LOG TRAIN {}] epoch: {:04}/{:04}, generator loss: {:.4f}'.format(now, epoch + 1, num_epochs, epoch_generator_loss))

    now = datetime.utcnow().strftime("%Y%m%d-%H_%M_%S")
    encoder_model_name = "{}_ep_{}_encoder_model.pth".format(now, (epoch+1))
    torch.save(encoder_train.state_dict(), os.path.join("./models", encoder_model_name))

    decoder_model_name = "{}_ep_{}_decoder_model.pth".format(now, (epoch+1))
    torch.save(decoder_train.state_dict(), os.path.join("./models", decoder_model_name))

    decoder_model_name = "{}_ep_{}_discriminator_model.pth".format(now, (epoch+1))
    torch.save(discriminator_train.state_dict(), os.path.join("./models", decoder_model_name))

In [ ]:
plt.plot(range(1, len(epoch_reconstruction_losses)+1), epoch_reconstruction_losses)

plt.title('AAE training performance')

plt.xlabel('training epochs')
plt.ylabel('reconstruction loss')

In [ ]:
plt.plot(range(0, len(epoch_discriminator_losses)), epoch_discriminator_losses)

plt.title('AENN training performance')

plt.xlabel('training epochs')
plt.ylabel('discrimination loss')

In [ ]:
plt.plot(range(0, len(epoch_generator_losses)), epoch_generator_losses)

plt.title('AENN training performance')

plt.xlabel('training epochs')
plt.ylabel('generation loss')

In [ ]:
encoder_eval = Encoder(input_size=ori_subset_transformed.shape[1], hidden_size=[256, 64, 16, 4, 2])
decoder_eval = Decoder(output_size=ori_subset_transformed.shape[1], hidden_size=[2, 4, 16, 64, 256])

if USE_CUDA:
    encoder_eval = encoder_eval.cuda()
    decoder_eval = decoder_eval.cuda()

encoder_eval.load_state_dict(torch.load('./models/20190818-03_22_18_ep_401_encoder_model.pth'))
decoder_eval.load_state_dict(torch.load('./models/20190818-03_22_18_ep_401_decoder_model.pth'))

encoder_eval.eval()
decoder_eval.eval()

torch_dataset = torch.from_numpy(ori_subset_transformed.values).float()

dataloader_eval = DataLoader(torch_dataset, batch_size=mini_batch_size, shuffle=False, num_workers=0)

if USE_CUDA:
    dataloader_eval = DataLoader(torch_dataset.cuda(), batch_size=mini_batch_size, shuffle=False)

In [ ]:
batch_count = 0

for enc_transactions_batch in dataloader_eval:
    z_enc_transactions_batch = encoder_eval(enc_transactions_batch)

    if batch_count == 0:
        z_enc_transactions_all = z_enc_transactions_batch
    else:
        z_enc_transactions_all = torch.cat((z_enc_transactions_all, z_enc_transactions_batch), dim=0)

    batch_count += 1

z_enc_transactions_all = z_enc_transactions_all.cpu().detach().numpy()

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)

regular_data = z_enc_transactions_all[label == 'regular']
global_outliers = z_enc_transactions_all[label == 'global']
local_outliers = z_enc_transactions_all[label == 'local']

ax.scatter(regular_data[:, 0], regular_data[:, 1], c='C0', marker="o", label='regular', edgecolors='w', linewidth=0.5)
ax.scatter(global_outliers[:, 0], global_outliers[:, 1], c='C1', marker="x", label='global', edgecolors='w', s=60)
ax.scatter(local_outliers[:, 0], local_outliers[:, 1], c='C3', marker="x", label='local', edgecolors='w', s=60)

ax.legend(loc='best')

In [ ]:
def compute_euclid_distance(x, y):
    euclidean_distance = np.sqrt(np.sum((x - y) ** 2, axis=1))

    return euclidean_distance

distances = np.apply_along_axis(func1d=compute_euclid_distance, axis=1, arr=z_enc_transactions_all, y=mu_gauss)

mode_divergence = np.min(distances, axis=1)

cluster_ids = np.argmin(distances, axis=1)

mode_divergence_all_scaled = np.asarray(mode_divergence)

for cluster_id in np.unique(cluster_ids).tolist():
    mask = cluster_ids == cluster_id
    mode_divergence_all_scaled[mask] = (mode_divergence[mask] - mode_divergence[mask].min()) / (mode_divergence[mask].ptp())

In [ ]:
plot_data = pd.concat([pd.Series(mode_divergence_all_scaled, name='mode_divergence'),
                       pd.Series(label, name='label'),
                       pd.Series(cluster_ids, name='cluster_id')],
                       axis=1)

num_clusters = len(np.unique(cluster_ids))

fig, axes = plt.subplots(1, num_clusters, sharey=True, figsize=(14, 10))

for mode in range(0, num_clusters):
    plot_data = plot_data.sample(frac=1.0)

    z_mode = plot_data[plot_data['cluster_id'] == mode]

    regular_data = z_mode[z_mode['label'] == 'regular']
    global_outliers = z_mode[z_mode['label'] == 'global']
    local_outliers = z_mode[z_mode['label'] == 'local']

    axes[mode].scatter(regular_data.index, regular_data['mode_divergence'],
                       c='C0', marker='o', s=30, linewidth=0.3, label='regular', edgecolors='w')

    axes[mode].scatter(global_outliers.index, global_outliers['mode_divergence'],
                       c='C1', marker='x', s=120, linewidth=3, label='global', edgecolors='w')

    axes[mode].scatter(local_outliers.index, local_outliers['mode_divergence'],
                       c='C3', marker='x', s=120, linewidth=3, label='local', edgecolors='w')

    xlabel = '$\\tau={}$' + str(mode+1) if mode == 0 else str(mode+1)
    axes[mode].set_xlabel(xlabel, fontsize=24)

    axes[mode].set_ylim([0.0, 1.1])

    axes[mode].set_xticks([int(plot_data.shape[0]/2)])
    axes[mode].set_xticklabels(['$x_{i}$'])

axes[0].set_ylabel('mode divergence $MD$', fontsize=20)

handles, labels = axes[2].get_legend_handles_labels()
plt.legend(handles, labels, loc='center', fontsize=20, ncol=3, borderaxespad=0.,
           bbox_to_anchor=(-6.5, 1., 9., .1))

plt.grid(True)

In [ ]:
reconstruction_criterion_categorical_eval = nn.BCEWithLogitsLoss(reduction='none')
reconstruction_criterion_numeric_eval = nn.MSELoss(reduction='none')

if USE_CUDA:
    reconstruction_criterion_categorical_eval = reconstruction_criterion_categorical_eval.cuda()
    reconstruction_criterion_numeric_eval = reconstruction_criterion_numeric_eval.cuda()

encoder_eval.eval()
decoder_eval.eval()

batch_count = 0

for enc_transactions_batch in dataloader_eval:
    z_enc_transactions_batch = encoder_eval(enc_transactions_batch)

    reconstruction_batch = decoder_eval(z_enc_transactions_batch)

    input_cat_all = enc_transactions_batch[:, :ori_dataset_categ_transformed.shape[1]]
    input_num_all = enc_transactions_batch[:, ori_dataset_categ_transformed.shape[1]:]

    rec_cat_all = reconstruction_batch[:, :ori_dataset_categ_transformed.shape[1]]
    rec_num_all = reconstruction_batch[:, ori_dataset_categ_transformed.shape[1]:]

    rec_error_cat_all = reconstruction_criterion_categorical_eval(input=rec_cat_all, target=input_cat_all).mean(dim=1)
    rec_error_num_all = reconstruction_criterion_numeric_eval(input=rec_num_all, target=input_num_all).mean(dim=1)

    rec_error_all_batch = rec_error_cat_all + rec_error_num_all

    if batch_count == 0:
        rec_error_all = rec_error_all_batch
    else:
        rec_error_all = torch.cat((rec_error_all, rec_error_all_batch), dim=0)

    batch_count += 1

rec_error_all = rec_error_all.cpu().detach().numpy()

rec_error_all_scaled = np.asarray(rec_error_all)

for cluster_id in np.unique(cluster_ids).tolist():
    mask = cluster_ids == cluster_id
    rec_error_all_scaled[mask] = (rec_error_all[mask] - rec_error_all[mask].min()) / (rec_error_all[mask].ptp())

In [ ]:
plot_data = pd.concat([pd.Series(rec_error_all_scaled, name='rec_error'),
                       pd.Series(label, name='label'),
                       pd.Series(cluster_ids, name='cluster_id')],
                       axis=1)

num_clusters = len(np.unique(cluster_ids))

fig, axes = plt.subplots(1, num_clusters, sharey=True, figsize=(14, 10))

for mode in range(0, num_clusters):
    plot_data = plot_data.sample(frac=1.0)

    z_mode = plot_data[plot_data['cluster_id'] == mode]

    regular_data = z_mode[z_mode['label'] == 'regular']
    global_outliers = z_mode[z_mode['label'] == 'global']
    local_outliers = z_mode[z_mode['label'] == 'local']

    axes[mode].scatter(regular_data.index, regular_data['rec_error'],
                       c='C0', marker='o', s=30, linewidth=0.3, label='regular', edgecolors='w')

    axes[mode].scatter(global_outliers.index, global_outliers['rec_error'],
                       c='C1', marker='x', s=120, linewidth=3, label='global', edgecolors='w')

    axes[mode].scatter(local_outliers.index, local_outliers['rec_error'],
                       c='C3', marker='x', s=120, linewidth=3, label='local', edgecolors='w')

    xlabel = '$\\tau={}$' + str(mode+1) if mode == 0 else str(mode+1)
    axes[mode].set_xlabel(xlabel, fontsize=24)

    axes[mode].set_ylim([0.0, 1.1])

    axes[mode].set_xticks([int(plot_data.shape[0]/2)])
    axes[mode].set_xticklabels(['$x_{i}$'])

axes[0].set_ylabel('reconstruction error $RE$', fontsize=20)

handles, labels = axes[2].get_legend_handles_labels()
plt.legend(handles, labels, loc='center', fontsize=20, ncol=3, borderaxespad=0.,
           bbox_to_anchor=(-6.5, 1., 9., .1))

plt.grid(True)

In [ ]:
alpha = 0.4

anomaly_score = alpha * rec_error_all_scaled + (1.0 - alpha) * mode_divergence_all_scaled

plot_data = pd.concat([pd.Series(anomaly_score, name='anomaly_score'),
                       pd.Series(label, name='label'),
                       pd.Series(cluster_ids, name='cluster_id')],
                       axis=1)

num_clusters = len(np.unique(cluster_ids))

fig, axes = plt.subplots(1, num_clusters, sharey=True, figsize=(14, 10))

for mode in range(0, num_clusters):
    plot_data = plot_data.sample(frac=1.0)

    z_mode = plot_data[plot_data['cluster_id'] == mode]

    regular_data = z_mode[z_mode['label'] == 'regular']
    global_outliers = z_mode[z_mode['label'] == 'global']
    local_outliers = z_mode[z_mode['label'] == 'local']

    axes[mode].scatter(regular_data.index, regular_data['anomaly_score'],
                       c='C0', marker='o', s=30, linewidth=0.3, label='regular', edgecolors='w')

    axes[mode].scatter(global_outliers.index, global_outliers['anomaly_score'],
                       c='C1', marker='x', s=120, linewidth=3, label='global', edgecolors='w')

    axes[mode].scatter(local_outliers.index, local_outliers['anomaly_score'],
                       c='C3', marker='x', s=120, linewidth=3, label='local', edgecolors='w')

    xlabel = '$\\tau={}$' + str(mode+1) if mode == 0 else str(mode+1)
    axes[mode].set_xlabel(xlabel, fontsize=24)

    axes[mode].set_ylim([0.0, 1.1])

    axes[mode].set_xticks([int(plot_data.shape[0]/2)])
    axes[mode].set_xticklabels(['$x_{i}$'])

axes[0].set_ylabel('anomaly score $AS$', fontsize=20)

handles, labels = axes[2].get_legend_handles_labels()
plt.legend(handles, labels, loc='center', fontsize=20, ncol=3, borderaxespad=0.,
           bbox_to_anchor=(-6.5, 1., 9., .1))

plt.grid(True)

In [ ]:
ori_dataset['label'] = label
ori_dataset['tau'] = cluster_ids
ori_dataset['a'] = anomaly_score

precision, recall = [], []
ths = []
da = 0
for i in range(num_clusters):
    maxregular = ori_dataset[(ori_dataset['label']=='regular') & (ori_dataset['tau']==i)]['a'].max()
    t1 = ori_dataset[(ori_dataset['label']!='regular') & (ori_dataset['tau']==i) & (ori_dataset['a']<=maxregular)]['a'].sort_values().reset_index(drop=True)
    t2 = ori_dataset[(ori_dataset['label']!='regular') & (ori_dataset['tau']==i) & (ori_dataset['a']>maxregular)]['a']
    ths.append(t1 if t1.shape[0] else [t2.min()] if t2.shape[0] else [1])
    da += t2.shape[0]

from itertools import product

for th in product(*ths):
    tp, fp, tn, fn = 0, 0, 0, 0
    for i in range(num_clusters):
        p = ori_dataset[(anomaly_score>=th[i]) & (cluster_ids==i)]
        n = ori_dataset[(anomaly_score<th[i]) & (cluster_ids==i)]
        tp += p[p['label']!='regular'].shape[0]
        fp += p[p['label']=='regular'].shape[0]
        tn += n[n['label']=='regular'].shape[0]
        fn += n[n['label']!='regular'].shape[0]
    precision.append(tp/(tp+fp) if tp else 1)
    recall.append(tp/(tp+fn))

total = ori_dataset[ori_dataset['label']!='regular'].shape[0]
for i in range(da + 1):
    precision.append(1)
    recall.append(i/total)

mf1 = 0
for p, r in zip(precision, recall):
    f1 = 2 * p * r / (p + r)
    if f1 > mf1:
        mf1 = f1
        mp, mr = p, r
print('precision: {:.4f}, recall: {:.4f}, f1 best: {:.4f}'.format(mp, mr, mf1))

data = pd.DataFrame({'precision':precision,'recall':recall})
sns.lineplot(x="recall", y="precision", data=data)
plt.xlim(0, 1.0)
plt.ylim(0, 1.05)